In [13]:
import math
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
import time
import numpy as np
import matplotlib.patches as patches


In [14]:
import ipywidgets as widgets
import os
from IPython.display import display_markdown

def copy_exercise(self):
    uname = txt_uname.value.replace(" ", "_").lower()
    #fpath = "~/MOLSPEC/local/"
    fpath = "../../../../local/"
    fname = "exercise8_" + uname + ".ipynb"
    
    if len(uname)<=0:
        print('Please enter a valid user name!')
    elif os.path.isfile(fpath+fname) and cb_overwrite.value==False:
        print('The file already exists! To overwrite check the \"Overwrite Existing\" box and try again.')
        FancyText = "Click [here](" + fpath + fname + ") to open existing copy."
        display_markdown(FancyText, raw=True)
    else:
        out = !{"cp exercise8.ipynb " + fpath+fname}
        if len(out)>0:
            for line in out:
                print(out)
        else:
            FancyText = "Successfully copied exercise to local directory!<br> Click [here](" + fpath + fname + ") to open."
            display_markdown(FancyText, raw=True)
    
txt_uname = widgets.Text(
    value='',
    placeholder='User name',
    description='Purdue ID:',
    disabled=False
)

bt_genfile = widgets.Button(
    description='Copy Exercise',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Enter your username and then click to create a local exercise file'
)

cb_overwrite = widgets.Checkbox(
    value=False,
    description='Overwrite Existing?',
    disabled=False
)

bt_genfile.on_click(copy_exercise)

display.display(widgets.HBox([txt_uname, bt_genfile, cb_overwrite]))

In [18]:
def run_all(ev):
    global go
    go = True
    display.display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))

button = widgets.Button(description="Run 2D!")
button.on_click(run_all)

go = False

display.display(button)

Button(description='Run 2D!', style=ButtonStyle())

NameError: name 'Javascript' is not defined

## Inhomogeneously Broadened ##

In [5]:

def calc_accel(x,efield):
    aExp = np.exp(-alpha*x)
    ax = - 2.0*Minv*De*alpha*aExp*(1.0 - aExp)
    ax += Qo*efield*Minv
    return ax

def vv_step(x,vx,ax,efield):
    if(Temp==0):
        axrand = 0
    else:
        axrand = math.sqrt(2.0*kB*Temp*gamma/dt)*np.random.normal(0,1,1)/M
    xnew = x + B*dt*vx + 0.5*B*dt*dt*(ax + axrand)
    axnew = calc_accel(xnew,efield)
    vxnew = A*vx + 0.5*dt*(A*ax + axnew + 2.0*B*axrand)
    return xnew,vxnew,axnew

def gauss_pulse(t):
    return np.cos(2.0*math.pi*(t-to)*nu)*np.exp(-((t-to)**2)/(2.0*sigma*sigma))

if go:
    
    to = 150e-15
    sigma = 10e-15
    nu = 50e+12

    tmax=400e-15      # Total simulation time in seconds
    tau2max = 50e-15
    dt=2e-15     # Time-step in seconds
    Nsteps=int((2.0*tmax+tau2max+3.0*sigma*2.355)/dt)
    M=12*(1.66054e-24)   # Mass in g
    Minv = 1.0/M
    Qo = 4.803e-10         # Elementary charge in statCoulombs
    taxis = np.arange(0,Nsteps)*dt   # Time axis (array of time steps)

    gamma = 500.0e-12  # grams/second
    kB = 1.38064852e-16                  # erg/K
    Temp = 0.0                           # K
    B = 1.0/(1.0 + 0.5*gamma*dt/M)
    A = B*(1.0 - 0.5*gamma*dt/M)

    wo = 6.28*50e12        # Oscillator (angular) frequency

    De = 1e+5*(kB*300.0)

    c = 2.9979e+10
    vaxis = np.fft.fftfreq(Nsteps)/(dt*c)

    v1 = 1000
    v2 = 2400

    ndx1 = np.where(np.abs(vaxis-v1)==np.min(np.abs(vaxis-v1)))[0][0]
    ndx2 = np.where(np.abs(vaxis-v2)==np.min(np.abs(vaxis-v2)))[0][0]

    Nmols = 500
    if Nmols>1:
        W = np.random.normal(wo, 0.1*wo, (Nmols))
    else:
        W = wo

    K = M*W**2
    alpha = np.sqrt(K/(2.0*De))
    def calc_response(Efield):
        X = np.zeros((Nmols))
        VX = np.zeros((Nmols))
        AX = calc_accel(X,Efield[0])
        Mu = np.zeros((Nsteps))
        for n in range(0,Nsteps):
            X,VX,AX = vv_step(X,VX,AX,Efield[n])
            Mu[n] = np.mean(X)*Qo
        return Mu

    tauStep1 = 6*1e-15
    tauStep2 = 4*1e-15
    Tau1 = np.arange(0.0e-15, tmax, tauStep1)
    Tau2 = np.arange(0.0e-15, tau2max, tauStep2)
    Emax = 50e+4
    scfac = 8e+26*(Emax/(50e+4))
    scfacTime = 4e+18*(Emax/(50e+4))
    for n2 in range(0, len(Tau2)):
        tau2 = Tau2[n2]
        for n1 in range(0, len(Tau1)):
            tau1 = Tau1[n1]

            E1 = Emax*gauss_pulse(taxis+tau1+tau2-tmax-3.0*sigma*2.355)
            E2 = Emax*gauss_pulse(taxis+tau2-tmax-3.0*sigma*2.355)
            E3 = Emax*gauss_pulse(taxis-tmax-3.0*sigma*2.355)

            Mu123 = calc_response(E1 + E2 + E3)
            Mu12 = calc_response(E1 + E2)
            Mu13 = calc_response(E1 + E3)
            Mu23 = calc_response(E2 + E3)
            Mu1 = calc_response(E1)
            Mu2 = calc_response(E2)
            Mu3 = calc_response(E3)

            Mu2D = Mu123 - Mu12 - Mu23 - Mu13 + Mu1 + Mu2 + Mu3
            fMu2D = np.fft.fft(Mu2D)
            fEprobe = np.fft.fft(E3)

            Sig = -np.imag(fMu2D/fEprobe)

            if (n1==0) and (n2==0):
                SpecMat = np.zeros((len(Tau1),len(Tau2),len(vaxis[ndx1:ndx2])))
                ppPts, = plt.plot(vaxis[ndx1:ndx2], Sig[ndx1:ndx2]*scfac)
                txt = plt.text(1750, 0.15, '$\\tau_1 = $'+str(int(tau1*1e+15))+' fs \n$\\tau_2$= '+str(int(tau2*1e+15))+' fs', fontsize=20)
                display.display(plt.gcf())
                display.clear_output(wait=True)

                ax2 = plt.axes([1.0,0.275,1,0.6])
                #fieldPts, = plt.plot(taxis, E1 + E2 + E3, 'k', linewidth=3)
                field1Pts, = plt.plot(taxis, E1/Emax, 'b')
                field2Pts, = plt.plot(taxis, E2/Emax, 'g')
                field3Pts, = plt.plot(taxis, E3/Emax, 'r')
                muPts, = plt.plot(taxis, Mu123, 'y', linewidth=3)
                plt.xticks([])
                plt.yticks([])
                plt.ylim([-1,1])
                plt.xlim([0,dt*Nsteps])
                plt.legend(['$E_1$', '$E_2$', '$E_3$', '$\\mu$'], fontsize=24, loc=(1.1,0.1))
            elif(n1%1==0):
                ppPts.set_ydata(Sig[ndx1:ndx2]*scfac)
                #fieldPts.set_ydata((E1+E2+E3)/(3.0*Emax))
                field1Pts.set_ydata((E1)/(Emax))
                field2Pts.set_ydata((E2)/(Emax))
                field3Pts.set_ydata((E3)/(Emax))
                muPts.set_ydata(Mu123*scfacTime)
                txt.set_text('$\\tau_1 = $'+str(int(tau1*1e+15))+' fs \n$\\tau_2$= '+str(int(tau2*1e+15))+' fs')
                display.display(plt.gcf())
                display.clear_output(wait=True)
            SpecMat[n1,n2,:] = Sig[ndx1:ndx2]

    fSpecMat = np.fft.fft(SpecMat, axis=1)
    vaxis_tau = np.fft.fftfreq(len(Tau2))/(c*tauStep2)
    window = (np.abs(vaxis_tau)<500)
    f2DMat = np.zeros(np.shape(fSpecMat))*1j
    for n1 in range(0, len(Tau1)):
        for n3 in range(0, len(vaxis[ndx1:ndx2])):
            f2DMat[n1,:,n3] = fSpecMat[n1,:,n3]*window

    Mat2D = np.real(np.fft.ifft(f2DMat, axis=1))
    Mat2D = np.fft.fft(f2DMat, axis=0)

    vaxis1 = np.fft.fftfreq(len(Tau1))/(c*tauStep1)
    v1 = 1200
    v2 = 2200

    a1 = np.where(np.abs(vaxis1-v1)==np.min(np.abs(vaxis1-v1)))[0][0]
    a2 = np.where(np.abs(vaxis1-v2)==np.min(np.abs(vaxis1-v2)))[0][0]
    vaxis1 = vaxis1[a1:a2]

    vaxis3 = vaxis[ndx1:ndx2]


    SigMat = Mat2D[a1:a2,0,:]
    print(np.shape(SigMat))
    print(np.shape(vaxis1))
    print(np.shape(vaxis3))

    plt.figure(3)
    plt.contourf(vaxis1,vaxis3,np.transpose(np.real(SigMat)), 25)

    plt.set_cmap(plt.cm.get_cmap('bwr'))
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.ylabel('$\\omega_3/2\pi c$', fontsize=20)
    plt.xlabel('$\\omega_1/2\pi c$', fontsize=20)

    # plt.ylim([1575,1825])
    plt.colorbar()
    plt.show()

## Homogeneously Broadened ##

In [ ]:

def calc_accel(x,efield):
    aExp = np.exp(-alpha*x)
    ax = - 2.0*Minv*De*alpha*aExp*(1.0 - aExp)
    ax += Qo*efield*Minv
    return ax

def vv_step(x,vx,ax,efield):
    if(Temp==0):
        axrand = 0
    else:
        axrand = math.sqrt(2.0*kB*Temp*gamma/dt)*np.random.normal(0,1,1)/M
    xnew = x + B*dt*vx + 0.5*B*dt*dt*(ax + axrand)
    axnew = calc_accel(xnew,efield)
    vxnew = A*vx + 0.5*dt*(A*ax + axnew + 2.0*B*axrand)
    return xnew,vxnew,axnew

def gauss_pulse(t):
    return np.cos(2.0*math.pi*(t-to)*nu)*np.exp(-((t-to)**2)/(2.0*sigma*sigma))

if go:

    to = 150e-15
    sigma = 10e-15
    nu = 50e+12

    tmax=500e-15      # Total simulation time in seconds
    tau2max = 50e-15
    dt=2e-15     # Time-step in seconds
    Nsteps=int((2.0*tmax+tau2max+3.0*sigma*2.355)/dt)
    M=12*(1.66054e-24)   # Mass in g
    Minv = 1.0/M
    Qo = 4.803e-10         # Elementary charge in statCoulombs
    taxis = np.arange(0,Nsteps)*dt   # Time axis (array of time steps)

    gamma = 750.0e-12  # grams/second
    kB = 1.38064852e-16                  # erg/K
    Temp = 0.0                           # K
    B = 1.0/(1.0 + 0.5*gamma*dt/M)
    A = B*(1.0 - 0.5*gamma*dt/M)

    wo = 6.28*50e12        # Oscillator (angular) frequency

    De = 1e+5*(kB*300.0)

    c = 2.9979e+10
    vaxis = np.fft.fftfreq(Nsteps)/(dt*c)

    v1 = 1000
    v2 = 2400

    ndx1 = np.where(np.abs(vaxis-v1)==np.min(np.abs(vaxis-v1)))[0][0]
    ndx2 = np.where(np.abs(vaxis-v2)==np.min(np.abs(vaxis-v2)))[0][0]

    Nmols = 1
    if Nmols>1:
        W = np.random.normal(wo, 0.1*wo, (Nmols))
    else:
        W = wo

    K = M*W**2
    alpha = np.sqrt(K/(2.0*De))
    def calc_response(Efield):
        X = np.zeros((Nmols))
        VX = np.zeros((Nmols))
        AX = calc_accel(X,Efield[0])
        Mu = np.zeros((Nsteps))
        for n in range(0,Nsteps):
            X,VX,AX = vv_step(X,VX,AX,Efield[n])
            Mu[n] = np.mean(X)*Qo
        return Mu

    tauStep1 = 6*1e-15
    tauStep2 = 4*1e-15
    Tau1 = np.arange(0.0e-15, tmax, tauStep1)
    Tau2 = np.arange(0.0e-15, tau2max, tauStep2)
    Emax = 50e+4
    scfac = 8e+26*(Emax/(50e+4))
    scfacTime = 4e+18*(Emax/(50e+4))
    for n2 in range(0, len(Tau2)):
        tau2 = Tau2[n2]
        for n1 in range(0, len(Tau1)):
            tau1 = Tau1[n1]

            E1 = Emax*gauss_pulse(taxis+tau1+tau2-tmax-3.0*sigma*2.355)
            E2 = Emax*gauss_pulse(taxis+tau2-tmax-3.0*sigma*2.355)
            E3 = Emax*gauss_pulse(taxis-tmax-3.0*sigma*2.355)

            Mu123 = calc_response(E1 + E2 + E3)
            Mu12 = calc_response(E1 + E2)
            Mu13 = calc_response(E1 + E3)
            Mu23 = calc_response(E2 + E3)
            Mu1 = calc_response(E1)
            Mu2 = calc_response(E2)
            Mu3 = calc_response(E3)

            Mu2D = Mu123 - Mu12 - Mu23 - Mu13 + Mu1 + Mu2 + Mu3
            fMu2D = np.fft.fft(Mu2D)
            fEprobe = np.fft.fft(E3)

            Sig = -np.imag(fMu2D/fEprobe)

            if (n1==0) and (n2==0):
                SpecMat = np.zeros((len(Tau1),len(Tau2),len(vaxis[ndx1:ndx2])))
                ppPts, = plt.plot(vaxis[ndx1:ndx2], Sig[ndx1:ndx2]*scfac)
                txt = plt.text(1750, 0.15, '$\\tau_1 = $'+str(int(tau1*1e+15))+' fs \n$\\tau_2$= '+str(int(tau2*1e+15))+' fs', fontsize=20)
                display.display(plt.gcf())
                display.clear_output(wait=True)

                ax2 = plt.axes([1.0,0.275,1,0.6])
                #fieldPts, = plt.plot(taxis, E1 + E2 + E3, 'k', linewidth=3)
                field1Pts, = plt.plot(taxis, E1/Emax, 'b')
                field2Pts, = plt.plot(taxis, E2/Emax, 'g')
                field3Pts, = plt.plot(taxis, E3/Emax, 'r')
                muPts, = plt.plot(taxis, Mu123, 'y', linewidth=3)
                plt.xticks([])
                plt.yticks([])
                plt.ylim([-1,1])
                plt.xlim([0,dt*Nsteps])
                plt.legend(['$E_1$', '$E_2$', '$E_3$', '$\\mu$'], fontsize=24, loc=(1.1,0.1))
            elif(n1%1==0):
                ppPts.set_ydata(Sig[ndx1:ndx2]*scfac)
                #fieldPts.set_ydata((E1+E2+E3)/(3.0*Emax))
                field1Pts.set_ydata((E1)/(Emax))
                field2Pts.set_ydata((E2)/(Emax))
                field3Pts.set_ydata((E3)/(Emax))
                muPts.set_ydata(Mu123*scfacTime)
                txt.set_text('$\\tau_1 = $'+str(int(tau1*1e+15))+' fs \n$\\tau_2$= '+str(int(tau2*1e+15))+' fs')
                display.display(plt.gcf())
                display.clear_output(wait=True)
            SpecMat[n1,n2,:] = Sig[ndx1:ndx2]

    fSpecMat = np.fft.fft(SpecMat, axis=1)
    vaxis_tau = np.fft.fftfreq(len(Tau2))/(c*tauStep2)
    window = (np.abs(vaxis_tau)<500)
    f2DMat = np.zeros(np.shape(fSpecMat))*1j
    for n1 in range(0, len(Tau1)):
        for n3 in range(0, len(vaxis[ndx1:ndx2])):
            f2DMat[n1,:,n3] = fSpecMat[n1,:,n3]*window

    Mat2D = np.real(np.fft.ifft(f2DMat, axis=1))
    Mat2D = np.fft.fft(f2DMat, axis=0)

    vaxis1 = np.fft.fftfreq(len(Tau1))/(c*tauStep1)
    v1 = 1200
    v2 = 2200

    a1 = np.where(np.abs(vaxis1-v1)==np.min(np.abs(vaxis1-v1)))[0][0]
    a2 = np.where(np.abs(vaxis1-v2)==np.min(np.abs(vaxis1-v2)))[0][0]
    vaxis1 = vaxis1[a1:a2]

    vaxis3 = vaxis[ndx1:ndx2]


    SigMat = Mat2D[a1:a2,0,:]
    print(np.shape(SigMat))
    print(np.shape(vaxis1))
    print(np.shape(vaxis3))

    plt.figure(3)
    plt.contourf(vaxis1,vaxis3,np.transpose(np.real(SigMat)), 25)

    plt.set_cmap(plt.cm.get_cmap('bwr'))
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.ylabel('$\\omega_3/2\pi c$', fontsize=20)
    plt.xlabel('$\\omega_1/2\pi c$', fontsize=20)

    # plt.ylim([1575,1825])
    plt.colorbar()
    plt.show()